In [1]:
import pandas as pd
import numpy as np
import pytz, sys,os
import plotly.express as px

sys.path.append(os.getcwd())

from datetime import datetime as dt
from datetime import timezone
from dateutil import tz
from src.config import SUBSCRIPTION_KEY, CUSTOMER_ID
from PF_API_Shared import get_all_plants, get_plant_devices, Dispatch,TimeInterval


ModuleNotFoundError: No module named 'src'

In [17]:

plant = 'Latitude'
start = dt(2022,8,1)
end = dt(2022,8,2)


In [18]:
dev_tags = [("Combiner", ["DC_CURRENT"]),
            ("Inverter", ["DC_CURRENT"]),
            ("Sensor Pyranometer POA", ["IRRADIANCE_POA"]),
                ("Meter",["AC_POWER"])]
  
rq_tags = []

plants = get_all_plants(SUBSCRIPTION_KEY, CUSTOMER_ID)  
plant_id = plants["id"][plants["name"] == plant].values[0]
dispatch = Dispatch(SUBSCRIPTION_KEY, '2096')
devices = get_plant_devices(SUBSCRIPTION_KEY,CUSTOMER_ID,plant_id)



In [19]:
for dtype, tags in dev_tags:
    dtype_devices = devices[devices['type']==dtype]
    
    df = dispatch.fetch_data(dtype_devices['id'], 
                                tags, 
                                query_range=TimeInterval(starttime=start,endtime=end))
    rq_tags.append(df)
inv_data = pd.concat(rq_tags, axis=1)
data = inv_data.copy(deep=True)

In [20]:
ident = [id for site, type, id, name in data.columns]
data.columns = ident


In [25]:
data = data.set_index([data.index])
data.index = pd.to_datetime(data.index)
data = data[~data.index.duplicated(keep='first')]
data.index = data.index.tz_convert(plant_tz)


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\dpinales\Anaconda3\envs\energyopt\lib\site-packages\IPython\core\interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\dpinales\AppData\Local\Temp\ipykernel_51340\2775631023.py", line 5, in <cell line: 5>
    plant_tz = timezone_convert(df)
  File "c:\Users\dpinales\Desktop\Arevbot\src\config.py", line 23, in timezone_convert
NameError: name 'df' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\dpinales\Anaconda3\envs\energyopt\lib\site-packages\IPython\core\interactiveshell.py", line 1993, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "c:\Users\dpinales\Anaconda3\envs\energyopt\lib\site-packages\IPython\core\ultratb.py", line 1118, in structured_traceback
    return FormattedTB.structured_traceback(
  File "c:\Users\dpinales\Anaconda3\envs\energyopt\lib\site

In [ ]:
"""For Filtered data, the data is filtered by the following parameters
        a. POA > 100
        b. (Meter Power > 1.5 MW) & (Meter Power < 25 MW)
        c. Hour > 12 (For shading purposes)
        d. All CBX's < 0 = 0
"""

data.loc[:, "Meter_Power"] = data.filter(regex="MTR").median(axis=1)
data.loc[:, "POA"] = data.filter(regex="MET").median(axis=1)

curtailment_limit = 499999

new_df = data.between_time("7:00", "17:00")
new_df = data[(data.loc[:,"POA"] > 100)&(data.loc[:,"Meter_Power"]<curtailment_limit*0.99)]
new_df[new_df<=0.00] = 0
day_df = new_df.resample("1D").sum()

day_df


In [11]:
path = "data\\Project_Metadata_tables.xlsx"
print(os.path.abspath(path))

dc_meta = pd.read_excel(path, sheet_name="Combiner_table").filter(items= ['plant_name', 'string_count','DC Rating Power kW', 'Module IMP'], axis=1)


c:\Users\dpinales\Desktop\Arevbot\Dataframe_mangement\data\Project_Metadata_tables.xlsx


FileNotFoundError: [Errno 2] No such file or directory: 'data\\Project_Metadata_tables.xlsx'

In [ ]:
day_df = day_df.filter(regex="CMB")

dc_issues = dc_meta[dc_meta["plant_name"]==plant]  
dc_rating = [i for i in dc_issues['DC Rating Power kW'].values]

relative_dc = []

max_dc = max(dc_rating)

for i in dc_rating:
    dc = (i/max_dc)
    relative_dc.append(dc)

string_count = [s for s in dc_issues['string_count']]

cmb_relative = day_df.div(relative_dc)

cmb_relative.loc[:,'norm'] = cmb_relative.apply(pd.Series.nlargest, axis = 1, n = 5).median(axis = 1)
t = cmb_relative.div(cmb_relative['norm'], axis = 0).clip(lower = 0, upper = 1)
t = t.iloc[:,:-1]

down_strings = (string_count*(1-t)).round()

down_strings.to_csv('latitude_DC.csv')


In [ ]:
e_downstrings = sum(int(down_strings[:]))
total_string = sum(int(string_count))
ratio_strings = e_downstrings/total_string

print(f"The estimated plants DC outage: {ratio_strings}")

In [ ]:
sum_of_cbx = cbx_data["Sum_of_CBX"]
sum_of_inv = inv_df["Sum_of_inv"]

def cmb_summation(a,b):
    total = (a/b)
    return total


In [ ]:
# cbx_data.to_csv(f"{plant} - August_Daily_Derates.csv")
cbx_data.to_csv(f"{plant} - API Data.csv")